## Install necessary libraries

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git@main
!pip install transformers==4.45.2

## Load and Preprocess Data, Create Base Whisper Model

In [2]:
#---------------------------Load Dataset--------------------------------------
from datasets import load_dataset, DatasetDict

# Load the full dataset
full_train = load_dataset("imonghose/bengali-asr-data", split="train")
full_test = load_dataset("imonghose/bengali-asr-data", split="test")



# Get dataset sizes
train_size = len(full_train)
test_size = len(full_test)

# Compute the original train-test ratio
original_ratio = test_size / train_size

# Compute the new test size corresponding to 60% train size
new_train_size = int(0.05 * train_size)
new_test_size = int(0.2 * test_size)

train_sample = full_train.shuffle(seed=42).select(range(new_train_size))
test_sample = full_test.shuffle(seed=42).select(range(new_test_size))

# Create a new dataset dictionary
bangla_dataset = DatasetDict({
    "train": train_sample,
    "test": test_sample
})


c:\IMON\Masters\DKE Course\Semester 3\HCNLP\FINE_TUNE_WHISPER\whisper-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.


In [3]:
#-----------Check dataset shape----------
bangla_dataset.shape

{'train': (7978, 3), 'test': (2042, 3)}

In [4]:
#-----------Check dataset structure----------
bangla_dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence', 'length'],
        num_rows: 7978
    })
    test: Dataset({
        features: ['audio', 'sentence', 'length'],
        num_rows: 2042
    })
})

In [5]:
#-----------Check dataset sample----------
bangla_dataset['test'][23]

{'audio': {'path': None,
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -3.05175781e-05,  0.00000000e+00,  0.00000000e+00]),
  'sampling_rate': 16000},
 'sentence': 'এরফলে বাংলাদেশ দল এশিয়ান গেমসে প্রথম স্বর্ণপদক লাভ করে।',
 'length': 6.12}

In [6]:
# -------------------Set model properties-----------------------------------
model_name_or_path = "openai/whisper-small"
language = "bengali"
task = "transcribe"

In [7]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path,language=language,task=task)

In [8]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path,language=language,task=task)

In [9]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)

In [10]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to('cuda')

In [11]:
#--------------------Downsample to 16 KHz--------------------------
from datasets import Audio

bangla_dataset = bangla_dataset.cast_column("audio", Audio(sampling_rate=16000))

In [12]:
#Check a test transcript
bangla_dataset['test'][7]['sentence']

'সালাহউদ্দিনের জীবনী লেখার জন্য তিনি খ্যাত।'

In [13]:
#check a train transcript
bangla_dataset['train'][16]['sentence']

'পদ্মা সেতু প্রকল্প পরিচালক'

In [14]:
#verify tokenization
tokenized_text = tokenizer(bangla_dataset['test'][2]['sentence'], return_tensors=None)
tokenized_text


{'input_ids': [50258, 50302, 50359, 50363, 29045, 237, 220, 29045, 116, 29045, 123, 156, 12811, 29045, 123, 29045, 250, 156, 100, 229, 220, 29045, 97, 29045, 123, 29045, 101, 29045, 123, 220, 29045, 227, 29045, 116, 29045, 122, 29045, 100, 29045, 122, 156, 12811, 29045, 96, 220, 29045, 243, 156, 100, 235, 156, 12811, 156, 11061, 29045, 94, 156, 15773, 29045, 122, 29045, 114, 156, 8667, 29045, 110, 156, 11061, 220, 29045, 103, 156, 100, 235, 156, 12811, 29045, 99, 156, 12811, 156, 100, 235, 29045, 114, 29045, 101, 156, 100, 229, 220, 29045, 116, 29045, 243, 156, 100, 235, 29045, 115, 29045, 106, 29045, 97, 29045, 122, 220, 29045, 99, 156, 100, 229, 29045, 244, 29045, 122, 29045, 101, 8703, 97, 50257], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [15]:
#------------------Custom preprocessing function for dataset-----------------------------
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch


In [16]:
#-------------------------Execute preprocessing---------------------------
bangla_dataset_processed = bangla_dataset.map(prepare_dataset, remove_columns=bangla_dataset.column_names["train"], num_proc=1)

In [17]:
#--------------------Prepossessed dataset structure-------------------
bangla_dataset_processed

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 7978
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 2042
    })
})

In [18]:
#--------------------Regenerate transcription from tokenized text-------------------
bangla_dataset_processed['test'][45]['labels']
target_text = tokenizer.decode(bangla_dataset_processed['test'][45]['labels'], skip_special_tokens=True)
target_text

'এটা রাষ্ট্রীয়ভাবে নির্ধারিত হয়ে থাকে।'

## Test predictions for base Whisper model

In [21]:
#------------------------------Check true vs prediction on few sentences before fine-tuning (Train set)------------------
import torch

for idx in range(3,7):
    # Get the tokenized target labels
    target_tokenized = bangla_dataset_processed["train"][idx]["labels"]

    # Decode the true text from tokenized format
    target_text = tokenizer.decode(target_tokenized, skip_special_tokens=True)

    # Convert input features to tensor and add batch dimension
    input_feature = torch.tensor(bangla_dataset_processed["train"][idx]["input_features"]).unsqueeze(0)

    # Ensure correct data type and move to GPU
    input_feature = input_feature.to(dtype=torch.float32, device='cuda')  # Use float16 if needed

    with torch.no_grad():
        op = model.generate(input_feature, language='bengali', task='transcribe')

    # Decode predicted text
    text_pred = tokenizer.batch_decode(op, skip_special_tokens=True)[0]

    print(f'-------{idx}------')
    print(f'True : {target_text} \nPred : {text_pred}')
    print('\n ')


-------3------
True : রমার স্বামী দিবানাথ সেন 
Pred :  Rāmārṣāmi divanātṣen

 
-------4------
True : বাংলাদেশের অভিজ্ঞ এ পেসার 
Pred :  बाग्णादिश रोभीक को एप प्रेसर

 
-------5------
True : ব্যাসদেব এক দিন 
Pred :  बेस्देप एक दिन

 
-------6------
True : আইসিসির অন্তত 
Pred :  I see here on total

 


In [22]:
#------------------------------Check true vs prediction on few sentences before fine-tuning (Test set) ------------------
import torch

for idx in range(3,7):
    # Get the tokenized target labels
    target_tokenized = bangla_dataset_processed["test"][idx]["labels"]

    # Decode the true text from tokenized format
    target_text = tokenizer.decode(target_tokenized, skip_special_tokens=True)

    # Convert input features to tensor and add batch dimension
    input_feature = torch.tensor(bangla_dataset_processed["test"][idx]["input_features"]).unsqueeze(0)

    # Ensure correct data type and move to GPU
    input_feature = input_feature.to(dtype=torch.float32, device='cuda')  # Use float16 if needed

    with torch.no_grad():
        op = model.generate(input_feature, language='bengali', task='transcribe')

    # Decode predicted text
    text_pred = tokenizer.batch_decode(op, skip_special_tokens=True)[0]

    print(f'-------{idx}------')
    print(f'True : {target_text} \nPred : {text_pred}')
    print('\n ')


-------3------
True : কলেজটি প্রতিষ্ঠা হবার পর নানান সময়ে সমালোচিত হয়েছে। 
Pred :  ្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្

 
-------4------
True : এটি পরিচালনা করেছিলেন তামিল চলচ্চিত্র শিল্পের পরিচালক মণি রত্নম। 
Pred :  ಸಿರಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿಕಿ

 
-------5------
True : এদের প্রাথমিক খাদ্য ইঁদুর জাতীয় ক্ষুদ্র-তীক্ষ্ণদন্ত প্রাণী। 
Pred :  एदे प्रात्मिक खाद्द, इदूर जातियो खुद्रो तिखनू दान्तो प्रानि

 
-------6------
True : পশ্চিমবঙ্গের জেলা ম্যাজিস্ট্রেটের তিনটি পদ এই কর্মকর্তাদের জন্য সংরক্ষিত। 
Pred :  ಸಾರಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲ

## Test Predictions for fine tuned Whisper model (LORA)

In [ ]:
#---------------------------------Load LORA model from Hugging Face Hub-----------------------------------

from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["k_proj", "v_proj", "q_proj", "out_proj"], lora_dropout=0.05, bias="none")


# Load base model
base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

peft_model = get_peft_model(base_model, config)

# Load LoRA adapter
final_loaded_model_lora = PeftModel.from_pretrained(peft_model, "imonghose/whisper-small-bengali-lora-final")
# final_loaded_model = PeftModel.from_pretrained(peft_model, "trained_model")

# ✅ Move model to GPU
final_loaded_model_lora = final_loaded_model_lora.to("cuda")


c:\IMON\Masters\DKE Course\Semester 3\HCNLP\FINE_TUNE_WHISPER\whisper-env\Lib\site-packages\peft\tuners\tuners_utils.py:168: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [25]:
#------------------------------Check true vs prediction on few sentences after fine-tuning (LORA) (Train set)------------------
import torch

for idx in range(3,7):
    # Get the tokenized target labels
    target_tokenized = bangla_dataset_processed["train"][idx]["labels"]

    # Decode the true text from tokenized format
    target_text = tokenizer.decode(target_tokenized, skip_special_tokens=True)

    # Convert input features to tensor and add batch dimension
    input_feature = torch.tensor(bangla_dataset_processed["train"][idx]["input_features"]).unsqueeze(0)

    # Ensure correct data type and move to GPU
    input_feature = input_feature.to(dtype=torch.float32, device='cuda')  # Use float16 if needed

    with torch.no_grad():
        op = final_loaded_model_lora.generate(input_feature, language='bengali', task='transcribe')

    # Decode predicted text
    text_pred = tokenizer.batch_decode(op, skip_special_tokens=True)[0]

    print(f'-------{idx}------')
    print(f'True : {target_text} \nPred : {text_pred}')
    print('\n ')


-------3------
True : রমার স্বামী দিবানাথ সেন 
Pred : ডমারসামে দিভানাতিসান

 
-------4------
True : বাংলাদেশের অভিজ্ঞ এ পেসার 
Pred : মাংনাদেশ্ষ্য়ভিক্ক এ প্রেসার

 
-------5------
True : ব্যাসদেব এক দিন 
Pred : ভেস দেপ একদেন

 
-------6------
True : আইসিসির অন্তত 
Pred : আয়িস্যর অন তত

 


In [27]:
#------------------------------Check true vs prediction on few sentences after fine-tuning (LORA) (Test set)------------------
import torch

for idx in range(3,7):
    # Get the tokenized target labels
    target_tokenized = bangla_dataset_processed["test"][idx]["labels"]

    # Decode the true text from tokenized format
    target_text = tokenizer.decode(target_tokenized, skip_special_tokens=True)

    # Convert input features to tensor and add batch dimension
    input_feature = torch.tensor(bangla_dataset_processed["test"][idx]["input_features"]).unsqueeze(0)

    # Ensure correct data type and move to GPU
    input_feature = input_feature.to(dtype=torch.float32, device='cuda')  # Use float16 if needed

    with torch.no_grad():
        op = final_loaded_model_lora.generate(input_feature, language='bengali', task='transcribe')

    # Decode predicted text
    text_pred = tokenizer.batch_decode(op, skip_special_tokens=True)[0]

    print(f'-------{idx}------')
    print(f'True : {target_text} \nPred : {text_pred}')
    print('\n ')


-------3------
True : কলেজটি প্রতিষ্ঠা হবার পর নানান সময়ে সমালোচিত হয়েছে। 
Pred : কলেস টিপতিসতা হাভার পর নানছমায় সমলেত দো হয়েছে

 
-------4------
True : এটি পরিচালনা করেছিলেন তামিল চলচ্চিত্র শিল্পের পরিচালক মণি রত্নম। 
Pred : এটি পরিচলানকরিছিলান তামিলছলছিতে সিলেরের পরিচলও মনীরাতনম

 
-------5------
True : এদের প্রাথমিক খাদ্য ইঁদুর জাতীয় ক্ষুদ্র-তীক্ষ্ণদন্ত প্রাণী। 
Pred : এদের প্ধমিক হাত্ত ইদুুর জাতিয় খুরে তিখ্যকন্ত পরানে

 
-------6------
True : পশ্চিমবঙ্গের জেলা ম্যাজিস্ট্রেটের তিনটি পদ এই কর্মকর্তাদের জন্য সংরক্ষিত। 
Pred : পুশিম গে জেলা মাইজেস্টেছ তিমতি পাতে কাল্ম কাদ্রদ্য়ে সন্রো ফীর্যব্যর

 


## Test Predictions for fine tuned Whisper model (LORA + SpecAugment)

In [36]:
#---------------------------------Load LORA+SpecAugment model from Hugging Face Hub-----------------------------------

from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["k_proj", "v_proj", "q_proj", "out_proj"], lora_dropout=0.05, bias="none")


# Load base model
base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

peft_model = get_peft_model(base_model, config)
peft_model = get_peft_model(peft_model, config)
# loaded_model = get_peft_model(loaded_model, "trained_model")

# Load LoRA adapter
final_loaded_model_lora_spec = PeftModel.from_pretrained(peft_model, "imonghose/whisper-small-bengali-lora-specaugment-final")

# ✅ Move model to GPU
final_loaded_model_lora_spec = final_loaded_model_lora_spec.to("cuda")


c:\IMON\Masters\DKE Course\Semester 3\HCNLP\FINE_TUNE_WHISPER\whisper-env\Lib\site-packages\peft\mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
c:\IMON\Masters\DKE Course\Semester 3\HCNLP\FINE_TUNE_WHISPER\whisper-env\Lib\site-packages\peft\mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'openai/whisper-small' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
c:\IMON\Masters\DKE Course\Semester 3\HCNLP\FINE_TUNE_WHISPER\whisper-env\Lib\site-packages\peft\tuners\tuners_utils.py:168: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [37]:
#--------Check true vs prediction on few sentences after fine-tuning (LORA+SpecAugment) (Train set)------------------
import torch

for idx in range(3,7):
    # Get the tokenized target labels
    target_tokenized = bangla_dataset_processed["train"][idx]["labels"]

    # Decode the true text from tokenized format
    target_text = tokenizer.decode(target_tokenized, skip_special_tokens=True)

    # Convert input features to tensor and add batch dimension
    input_feature = torch.tensor(bangla_dataset_processed["train"][idx]["input_features"]).unsqueeze(0)

    # Ensure correct data type and move to GPU
    input_feature = input_feature.to(dtype=torch.float32, device='cuda')  # Use float16 if needed

    with torch.no_grad():
        op = final_loaded_model_lora_spec.generate(input_feature, language='bengali', task='transcribe')

    # Decode predicted text
    text_pred = tokenizer.batch_decode(op, skip_special_tokens=True)[0]

    print(f'-------{idx}------')
    print(f'True : {target_text} \nPred : {text_pred}')
    print('\n ')


-------3------
True : রমার স্বামী দিবানাথ সেন 
Pred : ডমারসামে দিভানাতশান

 
-------4------
True : বাংলাদেশের অভিজ্ঞ এ পেসার 
Pred : মাংনাদেশ্ষ্য়ভিকব এ প্রেসার

 
-------5------
True : ব্যাসদেব এক দিন 
Pred : ভেস দেপ একদেন

 
-------6------
True : আইসিসির অন্তত 
Pred : আইসিস্য়র অনতত

 


In [38]:
#-------------Check true vs prediction on few sentences after fine-tuning (LORA+SpecAugment) (Test set)------------------
import torch

for idx in range(3,7):
    # Get the tokenized target labels
    target_tokenized = bangla_dataset_processed["test"][idx]["labels"]

    # Decode the true text from tokenized format
    target_text = tokenizer.decode(target_tokenized, skip_special_tokens=True)

    # Convert input features to tensor and add batch dimension
    input_feature = torch.tensor(bangla_dataset_processed["test"][idx]["input_features"]).unsqueeze(0)

    # Ensure correct data type and move to GPU
    input_feature = input_feature.to(dtype=torch.float32, device='cuda')  # Use float16 if needed

    with torch.no_grad():
        op = final_loaded_model_lora_spec.generate(input_feature, language='bengali', task='transcribe')

    # Decode predicted text
    text_pred = tokenizer.batch_decode(op, skip_special_tokens=True)[0]

    print(f'-------{idx}------')
    print(f'True : {target_text} \nPred : {text_pred}')
    print('\n ')


-------3------
True : কলেজটি প্রতিষ্ঠা হবার পর নানান সময়ে সমালোচিত হয়েছে। 
Pred : কলেসী পতিষতা হাভ্র পর নানছমায় সমলেত দোয়েছে

 
-------4------
True : এটি পরিচালনা করেছিলেন তামিল চলচ্চিত্র শিল্পের পরিচালক মণি রত্নম। 
Pred : এটি পর্যছলানকরিছিলান তামিলছলছিতের সিলেল্পর পরিচলও মনীরাতনম

 
-------5------
True : এদের প্রাথমিক খাদ্য ইঁদুর জাতীয় ক্ষুদ্র-তীক্ষ্ণদন্ত প্রাণী। 
Pred : এদের প্ধমিকাত্ত ইদুর জাতিয় খুর্যর তিখন্ত পরানে

 
-------6------
True : পশ্চিমবঙ্গের জেলা ম্যাজিস্ট্রেটের তিনটি পদ এই কর্মকর্তাদের জন্য সংরক্ষিত। 
Pred : পুশিম গে জেলা মাইজেশ্য়তেছ তিমতপাতে কাল্ম কাদ্য়্য় সঙন্র্পীর্য্যর

 


## Test Predictions for fine tuned Whisper model (BitFit)

In [39]:
#---------------------------------Load BitFit model from Hugging Face Hub-----------------------------------
final_loaded_model_bitfit = WhisperForConditionalGeneration.from_pretrained("imonghose/whisper-small-bengali-bitfit-final").to('cuda')

c:\IMON\Masters\DKE Course\Semester 3\HCNLP\FINE_TUNE_WHISPER\whisper-env\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\imong\.cache\huggingface\hub\models--imonghose--whisper-small-bengali-bitfit-final. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [40]:
#------------------------------Check true vs prediction on few sentences after fine-tuning (BitFit) (Train set)------------------
import torch

for idx in range(3,7):
    # Get the tokenized target labels
    target_tokenized = bangla_dataset_processed["train"][idx]["labels"]

    # Decode the true text from tokenized format
    target_text = tokenizer.decode(target_tokenized, skip_special_tokens=True)

    # Convert input features to tensor and add batch dimension
    input_feature = torch.tensor(bangla_dataset_processed["train"][idx]["input_features"]).unsqueeze(0)

    # Ensure correct data type and move to GPU
    input_feature = input_feature.to(dtype=torch.float32, device='cuda')  # Use float16 if needed

    with torch.no_grad():
        op = final_loaded_model_bitfit.generate(input_feature, language='bengali', task='transcribe')

    # Decode predicted text
    text_pred = tokenizer.batch_decode(op, skip_special_tokens=True)[0]

    print(f'-------{idx}------')
    print(f'True : {target_text} \nPred : {text_pred}')
    print('\n ')


-------3------
True : রমার স্বামী দিবানাথ সেন 
Pred : সামাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযা

 
-------4------
True : বাংলাদেশের অভিজ্ঞ এ পেসার 
Pred : সাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযা

 
-------5------
True : ব্যাসদেব এক দিন 
Pred : সালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালালা

 
-------6------
True : আইসিসির অন্তত 
Pred : আইসাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযাযা

In [41]:
#------------------------------Check true vs prediction on few sentences after fine-tuning (BitFit) (Test set)------------------
import torch

for idx in range(3,7):
    # Get the tokenized target labels
    target_tokenized = bangla_dataset_processed["test"][idx]["labels"]

    # Decode the true text from tokenized format
    target_text = tokenizer.decode(target_tokenized, skip_special_tokens=True)

    # Convert input features to tensor and add batch dimension
    input_feature = torch.tensor(bangla_dataset_processed["test"][idx]["input_features"]).unsqueeze(0)

    # Ensure correct data type and move to GPU
    input_feature = input_feature.to(dtype=torch.float32, device='cuda')  # Use float16 if needed

    with torch.no_grad():
        op = final_loaded_model_bitfit.generate(input_feature, language='bengali', task='transcribe')

    # Decode predicted text
    text_pred = tokenizer.batch_decode(op, skip_special_tokens=True)[0]

    print(f'-------{idx}------')
    print(f'True : {target_text} \nPred : {text_pred}')
    print('\n ')


-------3------
True : কলেজটি প্রতিষ্ঠা হবার পর নানান সময়ে সমালোচিত হয়েছে। 
Pred : কালেসেকামানালেকালেকালেকালেকালেকালে

 
-------4------
True : এটি পরিচালনা করেছিলেন তামিল চলচ্চিত্র শিল্পের পরিচালক মণি রত্নম। 
Pred : কাতেপেয়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়ায়

 
-------5------
True : এদের প্রাথমিক খাদ্য ইঁদুর জাতীয় ক্ষুদ্র-তীক্ষ্ণদন্ত প্রাণী। 
Pred : কানানানাকিকি ইদনিকিককা ইদিককি তানানানানা

 
-------6------
True : পশ্চিমবঙ্গের জেলা ম্যাজিস্ট্রেটের তিনটি পদ এই কর্মকর্তাদের জন্য সংরক্ষিত। 
Pred : কাসিনিনি জলা সিনা নান তাতা কাক ককা ককা ককা ককা ককা ককা ককা ককা ককা ককা ককা ককক ককা ককক ককক ককক ককক ককক ককক ককক কক কক কক কক কক কক কক কক কক কক কক কক ক কক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক ক 

 


## Test Predictions for fine tuned Whisper model (Adapter Layers)

In [42]:
#----------------------Create a custom Whisper class for injecting adapter layers----------------------
import torch
import torch.nn as nn
from transformers import WhisperForConditionalGeneration


class Adapter(nn.Module):
    def __init__(self, hidden_size, adapter_dim=64):
        super().__init__()
        self.down_proj = nn.Linear(hidden_size, adapter_dim)
        self.activation = nn.GELU()
        self.up_proj = nn.Linear(adapter_dim, hidden_size)

        # Xavier initialization
        nn.init.xavier_uniform_(self.down_proj.weight)
        nn.init.zeros_(self.down_proj.bias)
        nn.init.xavier_uniform_(self.up_proj.weight)
        nn.init.zeros_(self.up_proj.bias)

    def forward(self, x):
        return x + self.up_proj(self.activation(self.down_proj(x)))


class WhisperWithAdapters(WhisperForConditionalGeneration):
    def __init__(self, config, adapter_dim=64):
        super().__init__(config)
        self.adapter_dim = adapter_dim
        self._add_adapters()

    def _add_adapters(self):
        # Encoder adapters
        for layer in self.model.encoder.layers:
            hidden_size = layer.self_attn.embed_dim
            layer.adapter = Adapter(hidden_size, self.adapter_dim)

            original_forward = layer.forward

            def patched_forward(self, hidden_states, attention_mask=None, output_attentions=False, **kwargs):
                outputs = original_forward(
                    hidden_states,
                    attention_mask=attention_mask,
                    output_attentions=output_attentions,
                    **kwargs
                )
                hidden_states = outputs[0]
                hidden_states = self.adapter(hidden_states)
                return (hidden_states,) + outputs[1:]

            layer.forward = patched_forward.__get__(layer, type(layer))

        # Decoder adapters
        for layer in self.model.decoder.layers:
            hidden_size = layer.self_attn.embed_dim
            layer.adapter = Adapter(hidden_size, self.adapter_dim)

            original_forward = layer.forward

            def patched_forward(self, hidden_states, attention_mask=None, encoder_hidden_states=None,
                                encoder_attention_mask=None, output_attentions=False, **kwargs):
                outputs = original_forward(
                    hidden_states,
                    attention_mask=attention_mask,
                    encoder_hidden_states=encoder_hidden_states,
                    encoder_attention_mask=encoder_attention_mask,
                    output_attentions=output_attentions,
                    **kwargs
                )
                hidden_states = outputs[0]
                hidden_states = self.adapter(hidden_states)
                return (hidden_states,) + outputs[1:]

            layer.forward = patched_forward.__get__(layer, type(layer))

    def freeze_all_but_adapters(self):
        for param in self.parameters():
            param.requires_grad = False

        for name, module in self.named_modules():
            if 'adapter' in name:
                for param in module.parameters():
                    param.requires_grad = True

    def save_pretrained(self, save_directory, **kwargs):
        self.config.adapter_dim = self.adapter_dim
        super().save_pretrained(save_directory, **kwargs)

    @classmethod
    def from_pretrained(cls, pretrained_model_name_or_path, *model_args, config=None, **kwargs):
        from transformers import AutoConfig
        if config is None:
            config = AutoConfig.from_pretrained(pretrained_model_name_or_path)
        adapter_dim = getattr(config, "adapter_dim", 64)
        model = super().from_pretrained(pretrained_model_name_or_path, *model_args, config=config, **kwargs)
        model.__class__ = cls
        model.adapter_dim = adapter_dim
        model._add_adapters()
        return model

In [43]:
#---------------------------------Load Adapter Layers model from Hugging Face Hub-----------------------------------
from transformers import WhisperProcessor

final_loaded_model_adapter = WhisperWithAdapters.from_pretrained("imonghose/whisper-small-bengali-adapter-layer-final")

c:\IMON\Masters\DKE Course\Semester 3\HCNLP\FINE_TUNE_WHISPER\whisper-env\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\imong\.cache\huggingface\hub\models--imonghose--whisper-small-bengali-adapter-layer-final. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [45]:
# ✅ Move model to GPU
final_loaded_model_adapter = final_loaded_model_adapter.to("cuda")

In [46]:
#--------Check true vs prediction on few sentences after fine-tuning (Adapter Layers) (Train set)------------------
import torch

for idx in range(3,7):
    # Get the tokenized target labels
    target_tokenized = bangla_dataset_processed["train"][idx]["labels"]

    # Decode the true text from tokenized format
    target_text = tokenizer.decode(target_tokenized, skip_special_tokens=True)

    # Convert input features to tensor and add batch dimension
    input_feature = torch.tensor(bangla_dataset_processed["train"][idx]["input_features"]).unsqueeze(0)

    # Ensure correct data type and move to GPU
    input_feature = input_feature.to(dtype=torch.float32, device='cuda')  # Use float16 if needed

    with torch.no_grad():
        op = final_loaded_model_adapter.generate(input_feature, language='bengali', task='transcribe')

    # Decode predicted text
    text_pred = tokenizer.batch_decode(op, skip_special_tokens=True)[0]

    print(f'-------{idx}------')
    print(f'True : {target_text} \nPred : {text_pred}')
    print('\n ')


-------3------
True : রমার স্বামী দিবানাথ সেন 
Pred : 

 
-------4------
True : বাংলাদেশের অভিজ্ঞ এ পেসার 
Pred : 

 
-------5------
True : ব্যাসদেব এক দিন 
Pred : 

 
-------6------
True : আইসিসির অন্তত 
Pred : 

 


In [ ]:
#-------------Check true vs prediction on few sentences after fine-tuning (Adapter Layers) (Test set)------------------
import torch

for idx in range(3,7):
    # Get the tokenized target labels
    target_tokenized = bangla_dataset_processed["test"][idx]["labels"]

    # Decode the true text from tokenized format
    target_text = tokenizer.decode(target_tokenized, skip_special_tokens=True)

    # Convert input features to tensor and add batch dimension
    input_feature = torch.tensor(bangla_dataset_processed["test"][idx]["input_features"]).unsqueeze(0)

    # Ensure correct data type and move to GPU
    input_feature = input_feature.to(dtype=torch.float32, device='cuda')  # Use float16 if needed

    with torch.no_grad():
        op = final_loaded_model_adapter.generate(input_feature, language='bengali', task='transcribe')

    # Decode predicted text
    text_pred = tokenizer.batch_decode(op, skip_special_tokens=True)[0]

    print(f'-------{idx}------')
    print(f'True : {target_text} \nPred : {text_pred}')
    print('\n ')
